# Introduction to GeoPandas

## Preliminaries

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import contextily as cx
from geodatasets import get_path

## Reading and writing files
### Reading files

In [ ]:
path_to_data = get_path("nybb")
gdf = gpd.read_file(path_to_data)
gdf

### Writing Files

In [ ]:
gdf.to_file("my_file.geojon", driver="GeoJSON")

## Simple accessors and methods

### Measuring area

In [ ]:
gdf = gdf.set_index("BoroName")

In [ ]:
gdf['area'] = gdf.area
gdf['area']

### Getting polygon boundary and centroid

In [ ]:
gdf['boundary'] = gdf.boundary
gdf['boundary']

#### Comparison of polygon and boundary

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
gdf.plot(ax=ax[0], column='BoroCode', edgecolor='black')
gdf['boundary'].plot(ax=ax[1], edgecolor='red')

#### Another approach to comparison of polygons and boundaries

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
gdf.plot(ax=ax[0], column='BoroCode', edgecolor='black')
gdf.boundary.plot(ax=ax[1], edgecolor='red')

In [ ]:
gdf['centroid'] = gdf.centroid
gdf['centroid']

#### Centroids with polygons and boundaries

In [ ]:
m = gdf.plot(column='BoroCode', edgecolor='black', figsize=(12, 6))
gdf.boundary.plot(ax=m, edgecolor='red', linewidth=0.5)
gdf.centroid.plot(ax=m, color='red', markersize=20)

#### Centroids on dynamic map using explore

In [ ]:
m = gdf.explore(column='BoroCode', height=600, width=800)
gdf.centroid.explore(
    m=m,
    marker_kwds=dict(color='red', radius=5)
)
m

#### Centroids on dynamic map using Esri WorldImagery base map

In [ ]:
gdf["Name"] = gdf.index

m = gdf.explore(
    column="Name",  # make choropleth based on "BoroName" column
    tooltip="Name",  # show "BoroName" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    tiles="Esri WorldImagery",  # use "CartoDB positron" tiles
    cmap="Set1",  # use "Set1" matplotlib colormap
    style_kwds=dict(color="black"),
    legend=False,  # use black outline
    height=600,  # set height of map to 600 pixels
    width=800,  # set width of map to 800 pixels
)

gdf.centroid.explore(m=m)

#### Buffers around centroids on dynamic map using ESri WorldImagery base map

In [ ]:
buffers = gdf.centroid.buffer(2500)
buffers = gpd.GeoDataFrame(
    geometry=buffers,
    crs=gdf.crs,
    index=gdf.index
)

m = gdf.explore(
    column="Name",  # make choropleth based on "BoroName" column
    tooltip="Name",  # show "BoroName" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    tiles="Esri WorldImagery",  # use "CartoDB positron" tiles
    cmap="Set1",  # use "Set1" matplotlib colormap
    style_kwds=dict(color="black"),
    legend=False,  # use black outline
    width=800,  # set width of map to 800 pixels
    height=600  # set height of map to 600 pixels
)

buffers.explore(
    m=m
)

### Measuring distance

In [ ]:
first_point = gdf.centroid.iloc[0]
gdf['distance'] = gdf.centroid.distance(first_point)
gdf['distance']

In [ ]:
gdf['distance'].mean()

### Making maps

In [ ]:
gdf.set_geometry('geometry', inplace=True)
gdfw = gdf.to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(6, 6))
gdfw.plot(ax=ax,
          column='area',
          edgecolor='black',
          legend=True)
cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery, zoom=12)
ax.set_title('Boroughs of New York City')

In [ ]:
gdf.explore(column="area",
            tiles='Esri WorldImagery',
            legend=False,
            height=500,
            width=800,
            control_scale=True)

In [ ]:
gdfw['centroid'] = gdfw.centroid.to_crs(epsg=3857)
ax = gdfw.plot(column='area',
               edgecolor='black',
               legend=True,
               alpha=0.2)
gdfw['centroid'].plot(ax=ax,
                      color='red',
                      markersize=20)
cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery, zoom=12)

## Geometry creation
### Convex hull

In [ ]:
gdf['convex_hull'] = gdf.convex_hull

In [ ]:
ax = gdf['convex_hull'].plot(edgecolor='black', alpha=0.5, figsize=(12, 6))
gdf['boundary'].plot(ax=ax, edgecolor='red', linewidth=0.5)

### Buffer

In [ ]:
gdf['buffered'] = gdf.buffer(10_000)
gdf['buffered_centroid'] = gdf['centroid'].buffer(10_000)

ax = gdf['buffered'].plot(edgecolor='black', alpha=0.5)
gdf['buffered_centroid'].plot(
    ax=ax, color='red', edgecolor='red', linewidth=0.5)
gdf['boundary'].plot(ax=ax, edgecolor='blue', linewidth=0.5)
ax.set_title('Buffer of 10 km around borough centroids')

### Geometry relations

In [ ]:
brooklyn = gdf.loc["Brooklyn", "geometry"]
brooklyn

In [ ]:
type(brooklyn)

In [ ]:
gdf['buffered'].intersects(brooklyn)

In [ ]:
gdf['within'] = gdf['buffered_centroid'].within(gdf)
gdf = gdf.set_geometry("buffered_centroid")

ax = gdf.plot(
    "within",
    categorical=True,
    legend=True,
    alpha=0.5,
    legend_kwds={"loc": "upper left"}
)
gdf['boundary'].plot(ax=ax, color='black', linewidth=0.5)